# Apple Color Classification and Cropping Program

In [4]:
# Install necessary libraries
!pip install torch torchvision torchaudio
!pip install yolov5

In [5]:
# Import required libraries
import torch
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [6]:
# Step 1: Mount Google Drive to save images there
from google.colab import drive
drive.mount('/content/drive')

# Define the path to the folder in Google Drive
drive_folder_path = '/content/drive/My Drive/AppleImages'
# Create the folder if it doesn't exist
os.makedirs(drive_folder_path, exist_ok=True)

# Define the path to the local folder in Colab (for local storage)
local_folder_path = '/content/AppleImages'
# Create the folder if it doesn't exist
os.makedirs(local_folder_path, exist_ok=True)

Mounted at /content/drive


In [7]:
# Step 2: Load the YOLOv5 model with force_reload to ensure it's the latest version
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True)

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-7-19 Python-3.11.13 torch-2.6.0+cu124 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 97.1MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [8]:
# Step 3: Load the image to be processed from the URL
url = 'https://storage.googleapis.com/648010c1-f244-4641-98f2-73ff6c1b4e99/ai_assignment_20230726_classify.jpeg'
response = requests.get(url)
img = Image.open(BytesIO(response.content))

In [9]:
# Step 4: Perform object detection to identify apples
results = model(img)
results.print()  # Print results to see detected objects

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 1543x1943 26 apples
Speed: 105.0ms pre-process, 330.7ms inference, 40.7ms NMS per image at shape (1, 3, 512, 640)


In [10]:
# Step 5: Get the index of 'apple' in results.names
apple_index = list(results.names.keys())[list(results.names.values()).index('apple')]

In [11]:
# Step 6: Get bounding boxes for detected apples
apples = results.xywh[0][results.xywh[0][:, -1] == apple_index]

In [12]:
# Step 7: Define HSV ranges for red, green, and yellow apples
red_lower = np.array([0, 120, 70])
red_upper = np.array([10, 255, 255])
yellow_lower = np.array([20, 100, 100])
yellow_upper = np.array([30, 255, 255])
green_lower = np.array([35, 40, 40])
green_upper = np.array([85, 255, 255])

In [13]:
# Step 8: Function to classify apple color based on HSV ranges
def classify_apple_color(cropped_img):
    # Convert image to numpy array and to HSV color space
    cropped_array = np.array(cropped_img)
    cropped_hsv = cv2.cvtColor(cropped_array, cv2.COLOR_RGB2HSV)

    # Create masks for each color (red, yellow, green)
    mask_red = cv2.inRange(cropped_hsv, red_lower, red_upper)
    mask_yellow = cv2.inRange(cropped_hsv, yellow_lower, yellow_upper)
    mask_green = cv2.inRange(cropped_hsv, green_lower, green_upper)

    # Calculate the percentage of each color in the mask
    red_percentage = np.sum(mask_red) / mask_red.size
    yellow_percentage = np.sum(mask_yellow) / mask_yellow.size
    green_percentage = np.sum(mask_green) / mask_green.size

    # Classify apple color based on the highest percentage of color
    if red_percentage > max(yellow_percentage, green_percentage):
        return 'red'
    elif yellow_percentage > max(red_percentage, green_percentage):
        return 'yellow'
    else:
        return 'green'

In [14]:
# Step 9: Initialize counters for each color
color_counters = {'red': 1, 'yellow': 1, 'green': 1}

In [15]:
# Step 10: Process each detected apple, classify color, and save
for i, (*xywh, conf, cls) in enumerate(apples):
    # Extract coordinates for each detected apple
    x, y, w, h = map(float, xywh)

    # Crop the apple from the image based on bounding box coordinates
    cropped_img = img.crop((int(x - w/2), int(y - h/2), int(x + w/2), int(y + h/2)))
    cropped_img = cropped_img.convert('RGB')  # Convert to RGB format

    # Classify the color of the apple
    label = classify_apple_color(cropped_img)

    # Step 11: Save the cropped image to Google Drive
    drive_filename = f"{label}_{color_counters[label]}.jpg"
    drive_file_path = os.path.join(drive_folder_path, drive_filename)
    cropped_img.save(drive_file_path)

    # Step 12: Save the cropped image to local folder in Colab
    local_filename = f"{label}_{color_counters[label]}.jpg"
    local_file_path = os.path.join(local_folder_path, local_filename)
    cropped_img.save(local_file_path)

    # Step 13: Display the cropped image in Colab
    plt.imshow(cropped_img)
    plt.title(f"{label.capitalize()} Apple {color_counters[label]}")
    plt.axis('off')
    plt.show()

    # Step 14: Increment the counter for the apple color
    color_counters[label] += 1